In [128]:
import pandas as pd
import numpy as np
import requests
from matplotlib import cm
from matplotlib import colors

### Getting the dataset
We will scrape the website https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, to get the details of Toronto neighborhood. The data will be stored into a dataframe.

In [129]:
# define the url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Extract the tables
table_dfs = pd.read_html(url)

# Get the first table
table_df = table_dfs[0]
table_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Data preprocessing
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [130]:
values = ['Not assigned']
table_df.drop(pd.Index(np.where(table_df['Borough'].isin(values))[0]), inplace = True)
table_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now we can see the total number of rows.

In [131]:
print("There are {} number of rows in the dataframe.".format(table_df.shape[0]))

There are 103 number of rows in the dataframe.


### Assigning the latitude and longitude for each postal code

We will be using the csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data . This link gets downloaded with name 'Geospatial_Coordinates.csv'\
We can use pandas read_csv method to read the data into a dataframe.

In [132]:
geodata_df = pd.read_csv('Geospatial_Coordinates.csv')
geodata_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now the above dataframe needs to be merged with our postal code dataframe to get the full data.

In [133]:
table_df = table_df.join(geodata_df.set_index('Postal Code'), on='Postal Code')
table_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Explore the neighborhoods of Toronto using Foursquare API

We will be taking borough's that belongs to Toronto area.

In [134]:
toronto_df = table_df.loc[table_df['Borough'].str.contains('Toronto')]
toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
30,M4E,East Toronto,The Beaches,43.676357,-79.293031


Let us define the credentials to access Foursquare API's.

In [135]:
CLIENT_ID = 'MARBGB0XUSWGYWU2DBN2XNDIC2NJFECHSEP2MY10ZM5QZS2V' # your Foursquare ID
CLIENT_SECRET = 'RKZ0FXERT2WRFR4GRL1WSALH55VGWMXXL2GF0HLNY2GL3RRG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

We will write down a function to get the restaurants in the neighborhoods of Toronto within a radius of 500 meters.

In [136]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&section=food&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results ])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_restaurants*.

In [137]:
toronto_restaurants = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

In [138]:
toronto_restaurants.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
1,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
2,"Regent Park, Harbourfront",43.65426,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Brick Street Bakery,43.650574,-79.359539,Bakery
4,"Regent Park, Harbourfront",43.65426,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant


### Visualize the restaurants in Toronto neighborhood

In [139]:
import folium
map_restaurants = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_restaurants['Venue Latitude'], toronto_restaurants['Venue Longitude'], toronto_restaurants['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_restaurants)  
    
map_restaurants

Let's check how many venues/restaurants were returned for each neighborhood.

In [140]:
toronto_restaurants.groupby('Neighborhood').count()['Venue']

Neighborhood
Berczy Park                                                                                                   30
Brockton, Parkdale Village, Exhibition Place                                                                  15
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                           5
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     2
Central Bay Street                                                                                            30
Christie                                                                                                       7
Church and Wellesley                                                                                          30
Commerce Court, Victoria Hotel                                                                                30
Davisville                                                                         

In [141]:
toronto_restaurants_onehot = pd.get_dummies(toronto_restaurants['Venue Category'])
toronto_restaurants_onehot.insert(loc=0, column='Neighborhood', value=toronto_restaurants['Neighborhood'])

In [142]:
toronto_restaurants_onehot.head()

,Neighborhood,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,...,Steakhouse,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
toronto_restaurants_grouped = toronto_restaurants_onehot.groupby('Neighborhood').mean().reset_index()

In [144]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [145]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_restaurants_grouped['Neighborhood']

for ind in np.arange(toronto_restaurants_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_restaurants_grouped.iloc[ind, :], num_top_venues)

### Top 10 restaurants

In [146]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Bakery,Restaurant,French Restaurant,Café,Seafood Restaurant,Greek Restaurant,Diner,Italian Restaurant,Irish Pub,Indian Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Bakery,Sandwich Place,Burrito Place,Vietnamese Restaurant,Italian Restaurant,Japanese Restaurant,Restaurant,Diner
2,"Business reply mail Processing Centre, South C...",Fast Food Restaurant,Pizza Place,Restaurant,Burrito Place,Dumpling Restaurant,Creperie,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner
3,"CN Tower, King and Spadina, Railway Lands, Har...",American Restaurant,Tapas Restaurant,Ethiopian Restaurant,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
4,Central Bay Street,Café,Italian Restaurant,Japanese Restaurant,Chinese Restaurant,Sushi Restaurant,Sandwich Place,New American Restaurant,Portuguese Restaurant,Donut Shop,Diner
5,Christie,Café,American Restaurant,Diner,Italian Restaurant,Restaurant,Eastern European Restaurant,Deli / Bodega,Dim Sum Restaurant,Donut Shop,Dumpling Restaurant
6,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Restaurant,Italian Restaurant,Burger Joint,Mexican Restaurant,Pizza Place,Persian Restaurant,Burrito Place,Middle Eastern Restaurant
7,"Commerce Court, Victoria Hotel",Café,Restaurant,American Restaurant,Seafood Restaurant,Gastropub,Japanese Restaurant,Deli / Bodega,French Restaurant,Gluten-free Restaurant,New American Restaurant
8,Davisville,Pizza Place,Café,Sandwich Place,American Restaurant,Sushi Restaurant,Italian Restaurant,Asian Restaurant,Chinese Restaurant,Deli / Bodega,Diner
9,Davisville North,Pizza Place,Breakfast Spot,Asian Restaurant,Sandwich Place,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant


### Clustering the neighborhoods

In [116]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_restaurants_grouped_clustering = toronto_restaurants_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_restaurants_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
      dtype=int32)

In [118]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Café,Bakery,Restaurant,Breakfast Spot,Greek Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Deli / Bodega
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Sushi Restaurant,Burrito Place,Diner,Wings Joint,Burger Joint,Café,Chinese Restaurant,Creperie,Portuguese Restaurant,Falafel Restaurant
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0.0,Middle Eastern Restaurant,Café,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Ramen Restaurant,Restaurant,Ethiopian Restaurant,Gastropub,Diner
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,Café,Restaurant,Gastropub,Italian Restaurant,Japanese Restaurant,American Restaurant,Diner,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,New American Restaurant
30,M4E,East Toronto,The Beaches,43.676357,-79.293031,3.0,Pizza Place,Asian Restaurant,Burger Joint,Ethiopian Restaurant,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant


In [119]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(0)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Café,Bakery,Restaurant,Breakfast Spot,Greek Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Deli / Bodega
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Sushi Restaurant,Burrito Place,Diner,Wings Joint,Burger Joint,Café,Chinese Restaurant,Creperie,Portuguese Restaurant,Falafel Restaurant
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Middle Eastern Restaurant,Café,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Ramen Restaurant,Restaurant,Ethiopian Restaurant,Gastropub,Diner
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Restaurant,Gastropub,Italian Restaurant,Japanese Restaurant,American Restaurant,Diner,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,New American Restaurant
30,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Pizza Place,Asian Restaurant,Burger Joint,Ethiopian Restaurant,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant


### Visualize the resulting clusters

In [122]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining each cluster

In [123]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Regent Park, Harbourfront",Café,Bakery,Restaurant,Breakfast Spot,Greek Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Deli / Bodega
6,"Queen's Park, Ontario Provincial Government",Sushi Restaurant,Burrito Place,Diner,Wings Joint,Burger Joint,Café,Chinese Restaurant,Creperie,Portuguese Restaurant,Falafel Restaurant
13,"Garden District, Ryerson",Middle Eastern Restaurant,Café,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Ramen Restaurant,Restaurant,Ethiopian Restaurant,Gastropub,Diner
22,St. James Town,Café,Restaurant,Gastropub,Italian Restaurant,Japanese Restaurant,American Restaurant,Diner,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,New American Restaurant
31,Berczy Park,Bakery,Restaurant,French Restaurant,Café,Seafood Restaurant,Greek Restaurant,Diner,Italian Restaurant,Irish Pub,Indian Restaurant
40,Central Bay Street,Café,Italian Restaurant,Japanese Restaurant,Chinese Restaurant,Sushi Restaurant,Sandwich Place,New American Restaurant,Portuguese Restaurant,Donut Shop,Diner
41,Christie,Café,American Restaurant,Diner,Italian Restaurant,Restaurant,Eastern European Restaurant,Deli / Bodega,Dim Sum Restaurant,Donut Shop,Dumpling Restaurant
49,"Richmond, Adelaide, King",American Restaurant,Café,Seafood Restaurant,Steakhouse,Sushi Restaurant,Asian Restaurant,Breakfast Spot,Burrito Place,Brazilian Restaurant,Pizza Place
50,"Dufferin, Dovercourt Village",Pizza Place,Bakery,Portuguese Restaurant,Middle Eastern Restaurant,Café,Brazilian Restaurant,Eastern European Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner
58,"Harbourfront East, Union Station, Toronto Islands",Café,Fried Chicken Joint,Italian Restaurant,Restaurant,Pizza Place,New American Restaurant,Sandwich Place,Mexican Restaurant,Chinese Restaurant,Deli / Bodega


In [124]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,Lawrence Park,Dim Sum Restaurant,Wings Joint,Ethiopian Restaurant,Cuban Restaurant,Deli / Bodega,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Falafel Restaurant


In [125]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
147,Rosedale,Japanese Restaurant,Wings Joint,Falafel Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant


In [126]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,The Beaches,Pizza Place,Asian Restaurant,Burger Joint,Ethiopian Restaurant,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant
102,Davisville North,Pizza Place,Breakfast Spot,Asian Restaurant,Sandwich Place,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant


In [127]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
129,"Moore Park, Summerhill East",Italian Restaurant,Restaurant,Wings Joint,Eastern European Restaurant,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant
